## Free space 2

In this notebook we test against the previously obtained results for free space, save to 

        test2.py

In [1]:
from sympy import *
import numpy as np
import re

### Definitions and auxiliary functions

In [2]:
x1, y1, z1, x2, y2, z2 = symbols("x1, y1, z1, x2, y2, z2", real=True)
xij, yij, zij, dzij, zi = symbols("xij, yij, zij, dzij, zi", real=True)

b, h, eta = symbols("b, h, eta", real=True, positive=True)
theta = symbols("theta", real=True) ##eventually rename theta->psi


mu, nu1, nu2, gamma2, sigma, eta1, eta2, beta, alpha, tau, gamma1, kappa1, kappa2, gamma = symbols(
    "mu, nu1, nu2, gamma2, sigma, eta1, eta2, beta, alpha, tau, gamma1, kappa1, kappa2, gamma", integer=True)

R1 = Array([x1,y1,z1]) #why not use matrix here?
R2 = Array([x2,y2,z2])

KroneckerDelta = eye(3)

M  = Array([[1,0,0],[0,1,0],[0,0,-1]]) #diag(1,1,-1)

Mf = Array([[(1-theta)/(1+theta),0,0], [0, (1-theta)/(1+theta),0], [0,0,-1]])
#diag( (1-theta)/(1+theta), (1-theta)/(1+theta), -1) 

R2M = tensorcontraction(tensorproduct(M,R2), (1,2))

rM = R1 - R2M

rij = R1 - R2

def norm(r):
    result=0
    for i in range(3):
        result += r[i]**2
    return sqrt(result)#sqrt(summation(r[alpha]**2,(alpha,0,2)))

def eps(alpha, beta, gamma):
    return LeviCivita(alpha,beta,gamma)

In [3]:
def Laplacian(G, r):
    return diff(G, r[0], r[0]) + diff(G, r[1], r[1]) + diff(G, r[2], r[2])


## for some reason I cannot sum over 'symbols' here whenever rM is a Matrix, not an Array. Who knows why, but this seems to work. 

def Curl(mu, beta): ##Curl is always with respect to R1
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(mu,i,j)*diff(G(j,beta),R1[i]) ## Need dependence of G on rM and R2? 
    return result

def tilde_Curl(alpha, sigma): ##tilde_Curl is always with respect to R2
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(sigma,i,j)*diff(G(alpha, j),R2[i])
    return result

def CurlCurl(mu, sigma):
    result=0
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    result += eps(mu, i, j)*eps(sigma, k, l)*diff(G(j, l), R1[i], R2[k])
    return result

In [4]:
def Fl(G, l):
    return G + b**2/(4*l+6)*Laplacian(G, R1)

def tilde_Fl(G,l):
    return G + b**2/(4*l+6)*Laplacian(G, R2)

def FlFl(G, l, lprime):
    return G + b**2/(4*l+6)*Laplacian(G, R1) + b**2/(4*lprime+6)*Laplacian(G, R2) + b**4/((4*l+6)*(4*lprime+6))*Laplacian(Laplacian(G, R1), R2)

In [5]:
## Green's function
##first G* part, which is the same for ii and ij, later G0 part for ii

def G(alpha, beta):
    return 1/(8*pi*eta)*(KroneckerDelta[alpha,beta]/norm(rij) + rij[alpha]*rij[beta]/norm(rij)**3) ##can reuse for particle-particle interactions

## only for Gstar, particle - image interactions
# def Stokeslet(alpha, beta):
#     result = 0
#     for i in range(3):
#         result += Mf[beta, i]*GO(rM, alpha, i)
#     return result

# def Dipole(alpha, beta):
#     result = 0
#     for i in range(3):
#         result += -2*R2[2]*theta/(1+theta)*diff(GO(rM, alpha, 2) * M[beta, i], R1[i])
#     return result

# def Quadrupole(alpha, beta):
#     result = 0
#     for i in range(3):
#         result += R2[2]**2*theta/(1+theta)*Laplacian(GO(rM, alpha, i) * M[beta, i], R1)
#     return result

# def G(alpha, beta): #introduce flag to switch between G0 and Gstar, for whichever we have to compute the matrix elements, also include flag for wall, since theta=infty is hard to handle
#     return Stokeslet(alpha, beta) + Dipole(alpha, beta) + Quadrupole(alpha, beta)

In [6]:
## is there not a smarter way to do this? No dependence on rM and R2.. in python want xij, yij etc not x1- x2
## Need: self-interaction and ineraction with image. Interaction with other particle and with its image
## constant, R1=R2 (also constant), rij, and also rij. How to compute all of them with same code? 

## keep dependence on rM and R2: 
        ## for self-interaction choose rM = 0, 0, 2h and R2= 0,0,h
        ## for interaction choose: x1-x2=xij with particle, z1+z2 = ?

## keep dependence for now, might give greater flexibility later - no, delete to make shorter and write extra functions for particle-particle interactions
#for interactions, replace rM by xij, yij, dz = z1 + z2 or so, but there is also zij for particle-particle interactions, need another Greens function GO(alpha, beta)

## could use flag in G, whether to return Go or Gstar

In [7]:
def G1s1s(alpha, beta):
    return FlFl( G(alpha, beta), 0, 0)

def G2a1s(mu, beta):
    return b*tilde_Fl(Curl(mu, beta), 0)

def G1s2a(alpha,sigma):
    return 0.5*b*Fl(tilde_Curl(alpha, sigma), 0)

def G2a2a(mu, sigma):
    return 0.5*b**2*CurlCurl(mu, sigma)

def G1s2s(alpha, sigma, eta1):
    return 0.5*b*FlFl((diff(G(alpha,sigma), R2[eta1]) + diff(G(alpha, eta1), R2[sigma])), 0, 1)

def G2a2s(mu, sigma, eta1):
    return 0.5*b**2*tilde_Fl( (diff(Curl(mu,sigma), R2[eta1]) + diff(Curl(mu, eta1), R2[sigma])), 1)

def G2s1s(mu, nu1, beta):
    return 0.5*b*FlFl( diff( G(mu, beta), R1[nu1]) + diff( G(nu1, beta), R1[mu]), 1,0)

def G2s2a(mu, nu1, sigma):
    return 0.25*b**2*Fl( (diff(tilde_Curl(mu,sigma), R1[nu1]) + diff(tilde_Curl(nu1, sigma), R1[mu])), 1)

def G2s2s(mu, nu1, sigma, eta1):
    return 0.25*b**2*FlFl( diff( G(mu,sigma), R1[nu1], R2[eta1]) + diff( G(nu1,sigma), R1[mu], R2[eta1])
                          +  diff( G(mu,eta1), R1[nu1], R2[sigma]) + diff( G(nu1,eta1), R1[mu], R2[sigma]), 1,1)

def G2s3t(mu, nu1, eta2):
    return -0.1*b**3*Fl( Laplacian( diff( G(mu, eta2), R1[nu1]) + diff( G(nu1, eta2), R1[mu]), R2), 1)

def G2s3a(mu, nu1, sigma, eta2):
    return 1/3*b**3*Fl( diff( diff( tilde_Curl(mu, sigma), R2[eta2]) - 1/3*summation( eps(sigma, eta2, beta)*Laplacian( G(mu, beta), R2), (beta,0,2)), R1[nu1]) 
                       + diff( diff( tilde_Curl(nu1, sigma), R2[eta2]) - 1/3*summation( eps(sigma, eta2, beta)*Laplacian( G(nu1, beta), R2), (beta,0,2)), R1[mu]), 1)

def G3a2s(mu, nu2, sigma, nu1):
    return 0.25*b**3*tilde_Fl( diff( diff( Curl(tau, beta), R2[kappa1]) + diff( Curl(tau, kappa1), R2[beta]), R1[gamma2])
                              + diff( diff( Curl(gamma2, beta), R2[kappa1]) + diff( Curl(gamma2, kappa1), R2[beta]), R1[tau]), 1)

def G3a3a(mu, nu2, sigma, eta2):
    return 1/3*b**4*( diff( diff( CurlCurl(mu, sigma), R2[eta2]) - 1/3*summation( eps(sigma, eta2, beta)*Laplacian( Curl(mu, beta), R2), (beta,0,2)), R1[nu2]) 
                       + diff( diff( CurlCurl(nu2, sigma), R2[eta2]) - 1/3*summation( eps(sigma, eta2, beta)*Laplacian( Curl(nu2, beta), R2), (beta,0,2)), R1[mu]) )

def G1s3t(alpha, eta2):
    return -0.2*b**2*Fl( Laplacian( G(alpha, eta2), R2), 0)

def G1s3a(alpha, sigma, eta2):
    return 2/3*b**2*Fl( diff( tilde_Curl(alpha, sigma), R2[eta2]) - 1/3*summation(eps(sigma, eta2, beta)*Laplacian( G(alpha,beta), R2), (beta,0,2)), 0)

def G2a3a(mu, sigma, eta2):
    return 2/3*b**3*( diff( CurlCurl(mu, sigma), R2[eta2]) - 1/3*summation(eps(sigma, eta2, beta)*Laplacian( Curl(mu,beta), R2), (beta,0,2)) )

def G3a1s(mu, nu2, beta):
    return 0.5*b**2*tilde_Fl( diff( Curl(mu,beta), R1[nu2]) + diff( Curl(nu2,beta), R1[mu]), 0)

def G3a2a(mu, nu2, sigma):
    return 0.25*b**3*( diff( CurlCurl(mu,sigma), R1[nu2]) + diff( CurlCurl(nu2,sigma), R1[mu]) )


def g13sSym(alpha, sigma, eta1, eta2):
    return 1/3*( diff( G(alpha, sigma), R2[eta1], R2[eta2]) + diff( G(alpha, eta1), R2[eta2], R2[sigma]) + diff( G(alpha, eta2), R2[sigma], R2[eta1]) ) -  1/15*(
        KroneckerDelta[sigma, eta1]*Laplacian( G(alpha, eta2), R2) + KroneckerDelta[eta1, eta2]*Laplacian( G(alpha, sigma), R2)
        +KroneckerDelta[eta2, sigma]*Laplacian( G(alpha, eta1), R2) )

def G1s3s(alpha, sigma, eta1, eta2):
    return b**2*FlFl( g13sSym(alpha, sigma, eta1, eta2), 0, 2)

def G2a3s(mu, sigma, eta1, eta2):
    return b**3*tilde_Fl( 1/3*( diff( Curl(mu, sigma), R2[eta1], R2[eta2]) + diff( Curl(mu, eta1), R2[eta2], R2[sigma]) + diff( Curl(mu, eta2), R2[sigma], R2[eta1]) ) -  1/15*(
        KroneckerDelta[sigma, eta1]*Laplacian( Curl(mu, eta2), R2) + KroneckerDelta[eta1, eta2]*Laplacian( Curl(mu, sigma), R2)
        +KroneckerDelta[eta2, sigma]*Laplacian( Curl(mu, eta1), R2) ), 2)


def g3s1Sym(alpha, gamma1, gamma2, beta): ##screwed up alpha --> mu etc indexing, so caution with 3s modes below! 
    return 1/3*( diff( G(alpha, beta), R1[gamma1], R1[gamma2]) + diff( G(gamma1, beta), R1[gamma2], R1[alpha]) + diff( G(gamma2, beta), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( G(gamma2, beta), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( G(alpha, beta), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( G(gamma1, beta), R1) )

def G3s1s(mu, nu1, nu2, beta):
    return b**2*FlFl( g3s1Sym(mu, nu1, nu2, beta), 2,0)

def G3s2a(alpha, gamma1, gamma2, sigma):
    return 0.5*b**3*Fl( 1/3*( diff( tilde_Curl(alpha, sigma), R1[gamma1], R1[gamma2]) + diff( tilde_Curl(gamma1, sigma), R1[gamma2], R1[alpha]) 
                             + diff( tilde_Curl(gamma2, sigma), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( tilde_Curl(gamma2, sigma), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( tilde_Curl(alpha, sigma), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( tilde_Curl(gamma1, sigma), R1) ), 2)

def g2s1Sym(mu, nu1, sigma):
    return 0.5*(diff( G(mu, sigma), R1[nu1]) + diff( G(nu1, sigma), R1[mu]))

def G2s3s(mu, nu1, sigma, eta1, eta2):
    return b**3*FlFl( 1/3*( diff( g2s1Sym(mu, nu1, sigma), R2[eta1], R2[eta2]) + diff( g2s1Sym(mu, nu1, eta1), R2[eta2], R2[sigma]) 
                           + diff( g2s1Sym(mu, nu1, eta2), R2[sigma], R2[eta1]) ) -  1/15*(
        KroneckerDelta[sigma, eta1]*Laplacian( g2s1Sym(mu, nu1, eta2), R2) + KroneckerDelta[eta1, eta2]*Laplacian( g2s1Sym(mu, nu1, sigma), R2)
        +KroneckerDelta[eta2, sigma]*Laplacian( g2s1Sym(mu, nu1, eta1), R2) ), 1,2)

def g3a1sSym(mu, nu2, sigma):
    return 0.5*( diff( Curl(mu,sigma), R1[nu2]) + diff( Curl(nu2,sigma), R1[mu]) )

def G3a3s(mu, nu2, sigma, eta1, eta2):
    return b**4*tilde_Fl( 1/3*( diff( g3a1Sym(mu, nu2, sigma), R2[eta1], R2[eta2]) + diff( g3a1Sym(mu, nu2, eta1), R2[eta2], R2[sigma]) 
                           + diff( g3a1Sym(mu, nu2, eta2), R2[sigma], R2[eta1]) ) -  1/15*(
        KroneckerDelta[sigma, eta1]*Laplacian( g3a1Sym(mu, nu2, eta2), R2) + KroneckerDelta[eta1, eta2]*Laplacian( g3a1Sym(mu, nu2, sigma), R2)
        +KroneckerDelta[eta2, sigma]*Laplacian( g3a1Sym(mu, nu2, eta1), R2) ), 2)


def g12sSym(alpha, sigma, eta1):
    return 0.5*( diff(G(alpha,sigma), R2[eta1]) + diff(G(alpha, eta1), R2[sigma]) )

def G3s2s(alpha, gamma1, gamma2, sigma, eta1):
    return b**3*FlFl( 1/3*( diff( g12sSym(alpha, sigma, eta1), R1[gamma1], R1[gamma2]) + diff( g12sSym(gamma1, sigma, eta1), R1[gamma2], R1[alpha]) 
                           + diff( g12sSym(gamma2, sigma, eta1), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( g12sSym(gamma2, sigma, eta1), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( g12sSym(alpha, sigma, eta1), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( g12sSym(gamma1, sigma, eta1), R1) ), 2,1)


def g13aSym( alpha, sigma, eta2): 
    return diff( tilde_Curl(alpha, sigma), R2[eta2]) - 1/3*summation(eps(sigma, eta2, beta)*Laplacian( G(alpha,beta), R2), (beta,0,2))

def G3s3a(alpha, gamma1, gamma2, sigma, eta2):
    return 2/3*b**4*Fl( 1/3*( diff( g13aSym(alpha, sigma, eta2), R1[gamma1], R1[gamma2]) + diff( g13aSym(gamma1, sigma, eta2), R1[gamma2], R1[alpha]) 
                           + diff( g13aSym(gamma2, sigma, eta2), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( g13aSym(gamma2, sigma, eta2), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( g13aSym(alpha, sigma, eta2), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( g13aSym(gamma1, sigma, eta2), R1) ), 2)

def g13tSym(alpha, eta2):
    return Laplacian( G(alpha, eta2), R2)

def G3s3t(alpha, gamma1, gamma2, eta2):
    return -0.2*b**4*Fl( 1/3*( diff( g13tSym(alpha, eta2), R1[gamma1], R1[gamma2]) + diff( g13tSym(gamma1, eta2), R1[gamma2], R1[alpha]) 
                              + diff( g13tSym(gamma2, eta2), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( g13tSym(gamma2, eta2), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( g13tSym(alpha, eta2), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( g13tSym(gamma1, eta2), R1) ), 2)

def G3s3s(alpha, gamma1, gamma2, sigma, eta1, eta2):
    return b**4*FlFl( 1/3*( diff( g13sSym(alpha, sigma, eta1, eta2), R1[gamma1], R1[gamma2]) 
                           + diff( g13sSym(gamma1, sigma, eta1, eta2), R1[gamma2], R1[alpha]) 
                           + diff( g13sSym(gamma2, sigma, eta1, eta2), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( g13sSym(gamma2, sigma, eta1, eta2), R1) 
        + KroneckerDelta[gamma1, gamma2]*Laplacian( g13sSym(alpha, sigma, eta1, eta2), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( g13sSym(gamma1, sigma, eta1, eta2), R1) ), 2,2)

def G2a3t(mu, eta2):
    return -0.2*b**3*Laplacian( Curl(mu, eta2), R2)

def G3a3t(mu, nu2, eta2):
    return -0.2*b**4*Laplacian( g3a1sSym(mu, nu2, eta2), R2)


def k12sSym(alpha, sigma, eta1):
    return 0.5*( diff( G(alpha, eta1), R2[sigma]) + diff( G(alpha, sigma), R2[eta1]) )

def K1s2s(alpha, sigma, eta1):
    return 8*pi*eta*b**2/3*FlFl( k12sSym(alpha, sigma, eta1), 0,1)

def K1s3t(alpha, eta2):
    return -6*pi*eta*b**3/25*Fl( Laplacian( G(alpha, eta2), R2), 0)

def K2a2s(mu, sigma, eta1):
    return 4*pi*eta*b**3/3*tilde_Fl( diff( Curl(mu, eta1), R2[sigma]) + diff( Curl(mu, sigma), R2[eta1]), 1)


def k2s1Sym(mu, nu1, sigma):
    return 0.5*( diff( G(mu, sigma), R1[nu1]) + diff( G(nu1, sigma), R1[mu]))

def K2s2s(mu, nu1, sigma, eta1):
    return 4*pi*eta*b**3/3*FlFl( diff( k2s1Sym(mu, nu1, eta1), R2[sigma]) + diff( k2s1Sym(mu, nu1, sigma), R2[eta1]), 1,1)

def K2s3t(mu, nu1, eta2):
    return -6*pi*eta*b**4/25*Fl( 0.5*( diff( Laplacian(G(mu, eta2), R2), R1[nu1]) + diff( Laplacian(G(nu1, eta2), R2), R1[mu])), 1)

def K3a2s(mu, nu2, sigma, eta1):
    return 2*pi*eta*b**4/3*tilde_Fl( diff( diff( Curl(mu, eta1), R2[sigma]) + diff( Curl(mu, sigma), R2[eta1]), R1[nu2])
                                    + diff( diff( Curl(nu2, eta1), R2[sigma]) + diff( Curl(nu2, sigma), R2[eta1]), R1[mu]), 1)

def K3s2s(alpha, gamma1, gamma2, sigma, eta1): ## is this correct?
    return 8*pi*eta*b**4/3*FlFl( 1/3*( diff( k12sSym(alpha, sigma, eta1), R1[gamma1], R1[gamma2]) + diff( k12sSym(gamma1, sigma, eta1), R1[gamma2], R1[alpha]) 
                           + diff( k12sSym(gamma2, sigma, eta1), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( k12sSym(gamma2, sigma, eta1), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( k12sSym(alpha, sigma, eta1), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( k12sSym(gamma1, sigma, eta1), R1) ), 2,1) 

def K3s3t(alpha, gamma1, gamma2, eta2): ##up to factor same as G3s3t
    return -6*pi*eta*b**5/25*Fl( 1/3*( diff( g13tSym(alpha, eta2), R1[gamma1], R1[gamma2]) + diff( g13tSym(gamma1, eta2), R1[gamma2], R1[alpha]) 
                              + diff( g13tSym(gamma2, eta2), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( g13tSym(gamma2, eta2), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( g13tSym(alpha, eta2), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( g13tSym(gamma1, eta2), R1) ), 2)

def K2a3t(mu, eta2):
    return -6*pi*eta*b**4/25* Laplacian( Curl(mu, eta2), R2)

def K3a3t(mu, nu2, eta2):
    return -6*pi*eta*b**5/25*Laplacian( g3a1sSym(mu, nu2, eta2), R2)

### Systematic method of identifying which symmetry factor has to be applied

In [8]:
def check_duplicates(listOfElems): ## for eg 0,0,1 but will also give true for 0,0,0
    ''' Check if given list contains any duplicates '''
    if len(listOfElems) == len(set(listOfElems)):
        return False
    else:
        return True

def all_equal(iterator): ## for 0,0,0
    return len(set(iterator)) <= 1 



In [9]:
## 3s template
## Number of x,y and z: 18x, 6y and 3z. Want to stop if as quick as possible --> check x, then y, then z
m,n,l = 1,1,2

if check_duplicates([m,n,l]) and not all_equal([m,n,l]): ## two indices are equal
    print('x')
elif not check_duplicates([m,n,l]): ## all indices different
    print('y')
else:                           ## diagonal, all indices equal
    print('z') 

x


In [10]:
## 2s template (same for 3a)
## Number of a and b: 6a and 3b
i,j = 0,1

if not all_equal([i,j]): ## off-diagonal terms
    print('a')
else:                    ## diagonal terms
    print('b')


a


In [11]:
%%time
matG1s1s = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG1s1s[i,j] = G1s1s(i, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})

CPU times: user 6.6 s, sys: 11.9 ms, total: 6.61 s
Wall time: 6.61 s


In [12]:
%%time
matG1s1s = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG1s1s[i,j] = G1s1s(i, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})

CPU times: user 4.45 s, sys: 0 ns, total: 4.45 s
Wall time: 4.45 s


### Matrix assembly

In [13]:
%%time

matG1s1s = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG1s1s[i,j] = G1s1s(i, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
        
print('G1s1s done')
        
matG2a1s = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG2a1s[i,j] = G2a1s(i, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
        
print('G2a1s done')
        
matG1s2a = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG1s2a[i,j] = G1s2a(i, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
        
print('G1s2a done')
        
matG2a2a = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG2a2a[i,j] = G2a2a(i, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
        
print('G2a2a done')
        
matG1s2s = Matrix.zeros(3,6)
j=0
for i in range(3):
    for k in range(3):
        for l in range(k,3):
            if not all_equal([k,l]):
                matG1s2s[i,j] = 2*G1s2s(i, k, l).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                
            else:
                matG1s2s[i,j] = G1s2s(i, k, l).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
            j += 1
    j=0
    
print('G1s2s done')

matG2a2s = Matrix.zeros(3,6)
j=0
for i in range(3):
    for k in range(3):
        for l in range(k,3):
            if not all_equal([k,l]):
                matG2a2s[i,j] = 2*G2a2s(i, k, l).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                
            else:
                matG2a2s[i,j] = G2a2s(i, k, l).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
            j += 1
    j=0
    
print('G2a2s done')
    
matG2s1s = Matrix.zeros(6,3)
i=0
for j in range(3):
    for k in range(3):
        for l in range(k,3):
            if not all_equal([k,l]):
                matG2s1s[i,j] = 2*G2s1s(k, l, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                
            else:
                matG2s1s[i,j] = G2s1s(k, l, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
            i += 1
    i=0
    
print('G2s1s done')
    
matG2s2a = Matrix.zeros(6,3)
i=0
for j in range(3):
    for k in range(3):
        for l in range(k,3):
            if not all_equal([k,l]):
                matG2s2a[i,j] = 2*G2s2a(k, l, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                
            else:
                matG2s2a[i,j] = G2s2a(k, l, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
            i += 1
    i=0
    
print('G2s2a done')
    
matG2s2s = Matrix.zeros(6,6)
i=0
j=0
for ki in range(3):
    for li in range(ki,3):
        for kj in range(3):
            for lj in range(kj,3):                                          ##could also write as nested if statements -> done at level of (3s)
                if not all_equal([ki,li]) and not all_equal([kj,lj]):
                    matG2s2s[i,j] = 4*G2s2s(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                
                elif not all_equal([ki,li]) and all_equal([kj,lj]):
                    matG2s2s[i,j] = 2*G2s2s(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                elif all_equal([ki,li]) and not all_equal([kj,lj]):
                    matG2s2s[i,j] = 2*G2s2s(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                else:
                    matG2s2s[i,j] = G2s2s(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                j += 1
        j=0
        i += 1 
        
print('G2s2s done')
        
matG2s3t = Matrix.zeros(6,3)
i=0
for j in range(3):
    for k in range(3):
        for l in range(k,3):
            if not all_equal([k,l]):
                matG2s3t[i,j] = 2*G2s3t(k, l, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                
            else:
                matG2s3t[i,j] = G2s3t(k, l, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
            i += 1
    i=0
    
print('G2s3t done')
    
matG2s3a = Matrix.zeros(6,6)
i=0
j=0
for ki in range(3):
    for li in range(ki,3):
        for kj in range(3):
            for lj in range(kj,3):
                if not all_equal([ki,li]) and not all_equal([kj,lj]):
                    matG2s3a[i,j] = 4*G2s3a(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                
                elif not all_equal([ki,li]) and all_equal([kj,lj]):
                    matG2s3a[i,j] = 2*G2s3a(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                elif all_equal([ki,li]) and not all_equal([kj,lj]):
                    matG2s3a[i,j] = 2*G2s3a(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                else:
                    matG2s3a[i,j] = G2s3a(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                j += 1
        j=0
        i += 1 
        
print('G2s3a done')
        
matG3a2s = Matrix.zeros(6,6)         
i=0
j=0
for ki in range(3):
    for li in range(ki,3):
        for kj in range(3):
            for lj in range(kj,3):
                if not all_equal([ki,li]) and not all_equal([kj,lj]):
                    matG3a2s[i,j] = 4*G3a2s(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                
                elif not all_equal([ki,li]) and all_equal([kj,lj]):
                    matG3a2s[i,j] = 2*G3a2s(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                elif all_equal([ki,li]) and not all_equal([kj,lj]):
                    matG3a2s[i,j] = 2*G3a2s(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                else:
                    matG3a2s[i,j] = G3a2s(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                j += 1
        j=0
        i += 1 
        
print('G3a2s done')
        
matG3a3a = Matrix.zeros(6,6)
i=0
j=0
for ki in range(3):
    for li in range(ki,3):
        for kj in range(3):
            for lj in range(kj,3):
                if not all_equal([ki,li]) and not all_equal([kj,lj]):
                    matG3a3a[i,j] = 4*G3a3a(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                
                elif not all_equal([ki,li]) and all_equal([kj,lj]):
                    matG3a3a[i,j] = 2*G3a3a(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                elif all_equal([ki,li]) and not all_equal([kj,lj]):
                    matG3a3a[i,j] = 2*G3a3a(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                else:
                    matG3a3a[i,j] = G3a3a(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                j += 1
        j=0
        i += 1 
        
print('G3a3a done')
        
matK1s2s = Matrix.zeros(3,6)
j=0
for i in range(3):
    for k in range(3):
        for l in range(k,3):
            if not all_equal([k,l]):
                matK1s2s[i,j] = 2*K1s2s(i, k, l).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                
            else:
                matK1s2s[i,j] = K1s2s(i, k, l).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
            j += 1
    j=0
    
print('K1s2s done')
    
matK1s3t = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matK1s3t[i,j] = K1s3t(i, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
        
print('K1s3t done')
        
matK2a2s = Matrix.zeros(3,6)
j=0
for i in range(3):
    for k in range(3):
        for l in range(k,3):
            if not all_equal([k,l]):
                matK2a2s[i,j] = 2*K2a2s(i, k, l).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                
            else:
                matK2a2s[i,j] = K2a2s(i, k, l).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
            j += 1
    j=0  
    
print('K2a2s done')
    
matK2s2s = Matrix.zeros(6,6)
i=0
j=0
for ki in range(3):
    for li in range(ki,3):
        for kj in range(3):
            for lj in range(kj,3):
                if not all_equal([ki,li]) and not all_equal([kj,lj]):
                    matK2s2s[i,j] = 4*K2s2s(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                
                elif not all_equal([ki,li]) and all_equal([kj,lj]):
                    matK2s2s[i,j] = 2*K2s2s(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                elif all_equal([ki,li]) and not all_equal([kj,lj]):
                    matK2s2s[i,j] = 2*K2s2s(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                else:
                    matK2s2s[i,j] = K2s2s(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                j += 1
        j=0
        i += 1 
        
print('K2s2s done')
        
matK2s3t = Matrix.zeros(6,3)
i=0
for j in range(3):
    for k in range(3):
        for l in range(k,3):
            if not all_equal([k,l]):
                matK2s3t[i,j] = 2*K2s3t(k, l, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                
            else:
                matK2s3t[i,j] = K2s3t(k, l, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
            i += 1
    i=0
    
print('K2s3t done')
    
matK3a2s = Matrix.zeros(6,6)
i=0
j=0
for ki in range(3):
    for li in range(ki,3):
        for kj in range(3):
            for lj in range(kj,3):
                if not all_equal([ki,li]) and not all_equal([kj,lj]):
                    matK3a2s[i,j] = 4*K3a2s(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                
                elif not all_equal([ki,li]) and all_equal([kj,lj]):
                    matK3a2s[i,j] = 2*K3a2s(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                elif all_equal([ki,li]) and not all_equal([kj,lj]):
                    matK3a2s[i,j] = 2*K3a2s(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                else:
                    matK3a2s[i,j] = K3a2s(ki, li, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                j += 1
        j=0
        i += 1 
        
print('K3a2s done')
        
matG1s3t = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG1s3t[i,j] = G1s3t(i, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
        
print('G1s3t done')
        
matG1s3a = Matrix.zeros(3,6)
j=0
for i in range(3):
    for k in range(3):
        for l in range(k,3):
            if not all_equal([k,l]):
                matG1s3a[i,j] = 2*G1s3a(i, k, l).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                
            else:
                matG1s3a[i,j] = G1s3a(i, k, l).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
            j += 1
    j=0  
    
print('G1s3a done')
    
matG2a3a = Matrix.zeros(3,6)
j=0
for i in range(3):
    for k in range(3):
        for l in range(k,3):
            if not all_equal([k,l]):
                matG2a3a[i,j] = 2*G2a3a(i, k, l).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                
            else:
                matG2a3a[i,j] = G2a3a(i, k, l).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
            j += 1
    j=0  
    
print('G2a3a done')
    
matG3a1s = Matrix.zeros(6,3)
i=0
for j in range(3):
    for k in range(3):
        for l in range(k,3):
            if not all_equal([k,l]):
                matG3a1s[i,j] = 2*G3a1s(k, l, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                
            else:
                matG3a1s[i,j] = G3a1s(k, l, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
            i += 1
    i=0
    
print('G3a1s done')
    
matG3a2a = Matrix.zeros(6,3)
i=0
for j in range(3):
    for k in range(3):
        for l in range(k,3):
            if not all_equal([k,l]):
                matG3a2a[i,j] = 2*G3a2a(k, l, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                
            else:
                matG3a2a[i,j] = G3a2a(k, l, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
            i += 1
    i=0
    
print('G3a2a done')
    
matG1s3s = Matrix.zeros(3,10)
j=0
for i in range(3):
    for k in range(3):
        for l in range(k,3):
            for m in range(l,3):
                if check_duplicates([k,l,m]) and not all_equal([k,l,m]):
                    matG1s3s[i,j] = 3*G1s3s(rij,i, k, l, m).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                elif not check_duplicates([k,l,m]):
                    matG1s3s[i,j] = 6*G1s3s(rij,i, k, l, m).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                else:
                    matG1s3s[i,j] = G1s3s(rij,i, k, l, m).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                j += 1
    j=0
    
print('G1s3s done')
    
matG2a3s = Matrix.zeros(3,10)
j=0
for i in range(3):
    for k in range(3):
        for l in range(k,3):
            for m in range(l,3):
                if check_duplicates([k,l,m]) and not all_equal([k,l,m]):
                    matG2a3s[i,j] = 3*G2a3s(rij,i, k, l, m).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                elif not check_duplicates([k,l,m]):
                    matG2a3s[i,j] = 6*G2a3s(rij,i, k, l, m).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                else:
                    matG2a3s[i,j] = G2a3s(rij,i, k, l, m).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                j += 1
    j=0
    
print('G2a3s done')
    
matG3s1s = Matrix.zeros(10,3)
i=0
for j in range(3):
    for k in range(3):
        for l in range(k,3):
            for m in range(l,3):
                if check_duplicates([k,l,m]) and not all_equal([k,l,m]):
                    matG3s1s[i,j] = 3*G3s1s(k, l, m, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                elif not check_duplicates([k,l,m]):
                    matG3s1s[i,j] = 6*G3s1s(k, l, m, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                else:
                    matG3s1s[i,j] = G3s1s(k, l, m, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                i += 1
    i=0
    
print('G3s1s done')
    
matG3s2a = Matrix.zeros(10,3)
i=0
for j in range(3):
    for k in range(3):
        for l in range(k,3):
            for m in range(l,3):
                if check_duplicates([k,l,m]) and not all_equal([k,l,m]):
                    matG3s2a[i,j] = 3*G3s2a(k, l, m, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                elif not check_duplicates([k,l,m]):
                    matG3s2a[i,j] = 6*G3s2a(k, l, m, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                else:
                    matG3s2a[i,j] = G3s2a(k, l, m, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                i += 1
    i=0
    
print('G3s2a done')
    
matG2s3s = Matrix.zeros(6,10)
i=0
j=0
for ki in range(3):
    for li in range(ki,3):
        for kj in range(3):
            for lj in range(kj,3):
                for mj in range(lj,3):
                    if check_duplicates([kj,lj,mj]) and not all_equal([kj,lj,mj]):
                        if not all_equal([ki,li]):
                            matG2s3s[i,j] = 6*G2s3s(ki, li, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                        else:
                            matG2s3s[i,j] = 3*G2s3s(ki, li, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                                         
                    elif not check_duplicates([kj,lj,mj]):
                        if not all_equal([ki,li]):
                            matG2s3s[i,j] = 12*G2s3s(ki, li, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                        else:
                            matG2s3s[i,j] = 6*G2s3s(ki, li, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                                         
                    else:
                        if not all_equal([ki,li]):
                            matG2s3s[i,j] = 2*G2s3s(ki, li, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                        else:
                            matG2s3s[i,j] = G2s3s(ki, li, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})

                    j += 1
        j=0
        i += 1
        
print('G2s3s done')
        
matG3a3s = Matrix.zeros(6,10)
i=0
j=0
for ki in range(3):
    for li in range(ki,3):
        for kj in range(3):
            for lj in range(kj,3):
                for mj in range(lj,3):
                    if check_duplicates([kj,lj,mj]) and not all_equal([kj,lj,mj]):
                        if not all_equal([ki,li]):
                            matG3a3s[i,j] = 6*G3a3s(ki, li, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                        else:
                            matG3a3s[i,j] = 3*G3a3s(ki, li, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                                         
                    elif not check_duplicates([kj,lj,mj]):
                        if not all_equal([ki,li]):
                            matG3a3s[i,j] = 12*G3a3s(ki, li, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                        else:
                            matG3a3s[i,j] = 6*G3a3s(ki, li, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                                         
                    else:
                        if not all_equal([ki,li]):
                            matG3a3s[i,j] = 2*G3a3s(ki, li, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                        else:
                            matG3a3s[i,j] = G3a3s(ki, li, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})

                    j += 1
        j=0
        i += 1
        
print('G3a3s done')
        
matG3s2s = Matrix.zeros(10,6)
i=0
j=0
for ki in range(3):
    for li in range(ki,3):
        for mi in range(li,3):
            for kj in range(3):
                for lj in range(kj,3):
                    if check_duplicates([ki,li,mi]) and not all_equal([ki,li,mi]):
                        if not all_equal([kj,lj]):
                            matG3s2s[i,j] = 6*G3s2s(ki, li, mi, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                        else:
                            matG3s2s[i,j] = 3*G3s2s(ki, li, mi, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                                         
                    elif not check_duplicates([ki,li,mi]):
                        if not all_equal([kj,lj]):
                            matG3s2s[i,j] = 12*G3s2s(ki, li, mi, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                        else:
                            matG3s2s[i,j] = 6*G3s2s(ki, li, mi, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                                         
                    else:
                        if not all_equal([kj,lj]):
                            matG3s2s[i,j] = 2*G3s2s(ki, li, mi, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                        else:
                            matG3s2s[i,j] = G3s2s(ki, li, mi, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})

                    j += 1
            j=0
            i += 1
            
print('G3s2s done')
            
matG3s3a = Matrix.zeros(10,6)
i=0
j=0
for ki in range(3):
    for li in range(ki,3):
        for mi in range(li,3):
            for kj in range(3):
                for lj in range(kj,3):
                    if check_duplicates([ki,li,mi]) and not all_equal([ki,li,mi]):
                        if not all_equal([kj,lj]):
                            matG3s3a[i,j] = 6*G3s3a(ki, li, mi, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                        else:
                            matG3s3a[i,j] = 3*G3s3a(ki, li, mi, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                                         
                    elif not check_duplicates([ki,li,mi]):
                        if not all_equal([kj,lj]):
                            matG3s3a[i,j] = 12*G3s3a(ki, li, mi, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                        else:
                            matG3s3a[i,j] = 6*G3s3a(ki, li, mi, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                                         
                    else:
                        if not all_equal([kj,lj]):
                            matG3s3a[i,j] = 2*G3s3a(ki, li, mi, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                        else:
                            matG3s3a[i,j] = G3s3a(ki, li, mi, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})

                    j += 1
            j=0
            i += 1
            
print('G3s3t done')
            
matG3s3t = Matrix.zeros(10,3)
i=0
for j in range(3):
    for k in range(3):
        for l in range(k,3):
            for m in range(l,3):
                if check_duplicates([k,l,m]) and not all_equal([k,l,m]):
                    matG3s3t[i,j] = 3*G3s3t(k, l, m, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                elif not check_duplicates([k,l,m]):
                    matG3s3t[i,j] = 6*G3s3t(k, l, m, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                else:
                    matG3s3t[i,j] = G3s3t(k, l, m, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})

                i += 1
    i=0
    
print('G3s3t done')
    
matG3s3s = Matrix.zeros(10,10)
i=0
j=0
for ki in range(3):
    for li in range(ki,3):
        for mi in range(li,3):
            for kj in range(3):
                for lj in range(kj,3):
                    for mj in range(lj,3):
                        if check_duplicates([ki,li,mi]) and not all_equal([ki,li,mi]):
                            if check_duplicates([kj,lj,mj]) and not all_equal([kj,lj,mj]):
                                matG3s3s[i,j] = 9*G3s3s(ki, li, mi, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                                
                            elif not check_duplicates([kj,lj,mj]):
                                matG3s3s[i,j] = 18*G3s3s(ki, li, mi, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                                
                            else:
                                matG3s3s[i,j] = 3*G3s3s(ki, li, mi, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                                
                                
                        elif not check_duplicates([ki,li,mi]):
                            if check_duplicates([kj,lj,mj]) and not all_equal([kj,lj,mj]):
                                matG3s3s[i,j] = 18*G3s3s(ki, li, mi, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                                
                            elif not check_duplicates([kj,lj,mj]):
                                matG3s3s[i,j] = 36*G3s3s(ki, li, mi, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                                
                            else:
                                matG3s3s[i,j] = 6*G3s3s(ki, li, mi, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                                
                            
                        else:
                            if check_duplicates([kj,lj,mj]) and not all_equal([kj,lj,mj]):
                                matG3s3s[i,j] = 3*G3s3s(ki, li, mi, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                                
                            elif not check_duplicates([kj,lj,mj]):
                                matG3s3s[i,j] = 6*G3s3s(ki, li, mi, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                                
                            else:
                                matG3s3s[i,j] = G3s3s(ki, li, mi, kj, lj, mj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})

                        j += 1
            j=0
            i += 1
            
print('G3s3s done')

matK3s2s = Matrix.zeros(10,6)
i=0
j=0
for ki in range(3):
    for li in range(ki,3):
        for mi in range(li,3):
            for kj in range(3):
                for lj in range(kj,3):
                    if check_duplicates([ki,li,mi]) and not all_equal([ki,li,mi]):
                        if not all_equal([kj,lj]):
                            matK3s2s[i,j] = 6*K3s2s(ki, li, mi, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                        else:
                            matK3s2s[i,j] = 3*K3s2s(ki, li, mi, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                                         
                    elif not check_duplicates([ki,li,mi]):
                        if not all_equal([kj,lj]):
                            matK3s2s[i,j] = 12*K3s2s(ki, li, mi, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                        else:
                            matK3s2s[i,j] = 6*K3s2s(ki, li, mi, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                                         
                    else:
                        if not all_equal([kj,lj]):
                            matK3s2s[i,j] = 2*K3s2s(ki, li, mi, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                        else:
                            matK3s2s[i,j] = K3s2s(ki, li, mi, kj, lj).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})

                    j += 1
            j=0
            i += 1
            
print('K3s2s done')
            
matK3s3t = Matrix.zeros(10,3)
i=0
for j in range(3):
    for k in range(3):
        for l in range(k,3):
            for m in range(l,3):
                if check_duplicates([k,l,m]) and not all_equal([k,l,m]):
                    matK3s3t[i,j] = 3*K3s3t(k, l, m, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                elif not check_duplicates([k,l,m]):
                    matK3s3t[i,j] = 6*K3s3t(k, l, m, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})
                    
                else:
                    matK3s3t[i,j] = K3s3t(k, l, m, j).subs({x1-x2:xij, y1-y2:yij, z1-z2:zij})

                i += 1
    i=0
    
print('K3s3t done')
print('...done')

G1s1s done
G2a1s done
G1s2a done
G2a2a done
G1s2s done
G2a2s done
G2s1s done
G2s2a done
G2s2s done
G2s3t done
G2s3a done
G3a2s done
G3a3a done
K1s2s done
K1s3t done
K2a2s done
K2s2s done
K2s3t done
K3a2s done
G1s3t done
G1s3a done
G2a3a done
G3a1s done
G3a2a done


TypeError: G1s3s() takes 4 positional arguments but 5 were given

Create matrices that are zero

In [14]:
matG3t1s = Matrix(ZeroMatrix(3,3))
matG3t2a = Matrix(ZeroMatrix(3,3))

matG3t2s = Matrix(ZeroMatrix(3,6))
matG3t3t = Matrix(ZeroMatrix(3,3))

matG3t3a = Matrix(ZeroMatrix(3,6))
matG3t3s = Matrix(ZeroMatrix(3,10))

matK3t2s = Matrix(ZeroMatrix(3,6))
matK3t3t = Matrix(ZeroMatrix(3,3))

matK3t3a = Matrix(ZeroMatrix(3,6))
matK3t3s = Matrix(ZeroMatrix(3,10))

matG2a3t = Matrix(ZeroMatrix(3,3))
matG3a3t = Matrix(ZeroMatrix(6,3))

matK2a3t = Matrix(ZeroMatrix(3,3))
matK3a3t = Matrix(ZeroMatrix(6,3))

Need the following objects

GH1s(force), GH2a(torque), KHH(VH), GHH(FH), 

G1s1s(force), G2a1s(force), G1s2a(torque), G2a2a(torque)
G1sH(FH), K1sH(VH), G2aH(FH), K2aH(FH)

Want to assemble them locally and then print them to a py file. keep placeholders such as force (array of len 3), FH (array of len 20) etc. create these first...

In [15]:
## In python code will be force[i,0], so have to create force as np.zeros([3*Np,1]) object instead of np.zeros([3*Np])
force  = Matrix(MatrixSymbol('force',3,1)) ##create an actual Matrix from it, have to pass to 'Matrix' 
torque = Matrix(MatrixSymbol('torque',3,1))
VH     = Matrix(MatrixSymbol('VH',25,1))
FH     = Matrix(MatrixSymbol('FH',25,1))

Create block matrices, then multiply with vectors and then print as functions

In [16]:
matG1sH = Matrix(BlockMatrix([matG1s2s, matG1s3t, matG1s3a, matG1s3s]))
matG2aH = Matrix(BlockMatrix([matG2a2s, matG2a3t, matG2a3a, matG2a3s]))

matK1sH = Matrix(BlockMatrix([matK1s2s, matK1s3t, Matrix(ZeroMatrix(3,16))]))
matK2aH = Matrix(BlockMatrix([matK2a2s, matK2a3t, Matrix(ZeroMatrix(3,16))]))

matGH1s = Matrix(BlockMatrix([[matG2s1s],
                              [matG3t1s],
                              [matG3a1s],
                              [matG3s1s]]))
matGH2a = Matrix(BlockMatrix([[matG2s2a],
                              [matG3t2a],
                              [matG3a2a],
                              [matG3s2a]]))

matGHH = Matrix(BlockMatrix([[matG2s2s, matG2s3t, matG2s3a, matG2s3s],
                             [matG3t2s, matG3t3t, matG3t3a, matG3t3s],
                             [matG3a2s, matG3a3t, matG3a3a, matG3a3s],
                             [matG3s2s, matG3s3t, matG3s3a, matG3s3s]]))

matKHH_nonzero = Matrix(BlockMatrix([[matK2s2s, matK2s3t],
                                     [matK3t2s, matK3t3t],
                                     [matK3a2s, matK3a3t],
                                     [matK3s2s, matK3s3t]]))
matKHH = Matrix(BlockMatrix([matKHH_nonzero, Matrix(ZeroMatrix(25,16))]))

##diagonal elements
g1s = 1/(6*pi*eta*b)
g2a = 1/(4*pi*eta*b)

g2s = 3/(20*pi*eta*b)
g3t = 1/(2*pi*eta*b)
g3a = 3/(2*pi*eta*b)
g3s = 6/(7*pi*eta*b)

matGoHH = diag(g2s,g2s,g2s,g2s,g2s,g2s, g3t,g3t,g3t, g3a,g3a,g3a,g3a,g3a,g3a, g3s,g3s,g3s,g3s,g3s,g3s,g3s,g3s,g3s,g3s)

halfMinusk2s = 0.6
halfMinusk3t = 0.4
halfMinusk3a = 0.8
halfMinusk3s = 19/35

matKoHH = diag(halfMinusk2s,halfMinusk2s,halfMinusk2s,halfMinusk2s,halfMinusk2s,halfMinusk2s,
               halfMinusk3t,halfMinusk3t,halfMinusk3t,
               halfMinusk3a,halfMinusk3a,halfMinusk3a,halfMinusk3a,halfMinusk3a,halfMinusk3a,
               halfMinusk3s,halfMinusk3s,halfMinusk3s,halfMinusk3s,halfMinusk3s,halfMinusk3s,halfMinusk3s,halfMinusk3s,halfMinusk3s,halfMinusk3s)

NameError: name 'matG2a3s' is not defined

Create vectors instead of constructing the full matrix

In [ ]:
vecGH1sF = matGH1s*force
vecGH2aT = matGH2a*torque

vecKHHVH_diag = matKoHH*VH
vecKHHVH = matKHH*VH

vecGHHFH_diag = matGoHH*FH
vecGHHFH = matGHH*FH

vecG1s1sF = matG1s1s*force
vecG2a1sF = matG2a1s*force

vecG1s2aT = matG1s2a*torque
vecG2a2aT = matG2a2a*torque

vecG1sHFH = matG1sH*FH
vecG2aHFH = matG2aH*FH

vecK1sHVH = matK1sH*VH
vecK2aHVH = matK2aH*VH

_____________

### Write symbolic results to file as reusable python functions

In [ ]:
ME_file = "test2.py" #"freeSpaceME.py"

with open(ME_file, "w") as text_file:
    print("import numpy", file=text_file)
    print("PI = 3.14159265359\n", file=text_file)
    
    print("def G1s1sF(xij,yij,zij, b,eta, force):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG1s1sF),(3,)).tolist()), file=text_file)
    print("def G2a1sF(xij,yij,zij, b,eta, force):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2a1sF),(3,)).tolist()), file=text_file)
    
    print("def G1s2aT(xij,yij,zij, b,eta, torque):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG1s2aT),(3,)).tolist()), file=text_file)
    print("def G2a2aT(xij,yij,zij, b,eta, torque):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2a2aT),(3,)).tolist()), file=text_file)
    
    print("def GH1sF(xij,yij,zij, b,eta, force):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecGH1sF),(25,)).tolist()), file=text_file)
    print("def GH2aT(xij,yij,zij, b,eta, torque):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecGH2aT),(25,)).tolist()), file=text_file)
    
    print("def KoHHVH(b,eta, VH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecKHHVH_diag),(25,)).tolist()), file=text_file)
    print("def KHHVH(xij,yij,zij, b,eta, VH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecKHHVH),(25,)).tolist()), file=text_file)
    
    print("def GoHHFH(b,eta, FH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecGHHFH_diag),(25,)).tolist()), file=text_file)
    print("def GHHFH(xij,yij,zij, b,eta, FH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecGHHFH),(25,)).tolist()), file=text_file)
    
    print("def G1sHFH(xij,yij,zij, b,eta, FH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG1sHFH),(3,)).tolist()), file=text_file)
    print("def G2aHFH(xij,yij,zij, b,eta, FH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2aHFH),(3,)).tolist()), file=text_file)
    
    print("def K1sHVH(xij,yij,zij, b,eta, VH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecK1sHVH),(3,)).tolist()), file=text_file)
    print("def K2aHVH(xij,yij,zij, b,eta, VH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecK2aHVH),(3,)).tolist()), file=text_file)
    
    
    
## replace things for easier usage with python     
with open(ME_file, 'r') as file:
    filedata = file.read()
    
filedata = filedata.replace('pi', 'PI')
filedata = filedata.replace('sqrt', 'numpy.sqrt')

filedata = re.sub("force\[(\d), 0\]", r"force[\1]", filedata)
filedata = re.sub("torque\[(\d), 0\]", r"torque[\1]", filedata)
filedata = re.sub("VH\[(\d||\d\d), 0\]", r"VH[\1]", filedata)
filedata = re.sub("FH\[(\d||\d\d), 0\]", r"FH[\1]", filedata)

with open(ME_file, 'w') as file:
    file.write(filedata)

______________________________________
_______________________